In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain.llms import Ollama

In [2]:
db_path = "data/sample.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
llm = Ollama(base_url="http://localhost:11434",model="project")
llm.invoke("Who are you")
print(db.get_usable_table_names())

['maintenance', 'universityconfigurations']


In [6]:
from vanna.ollama import Ollama
from vanna.chromadb import ChromaDB_VectorStore

class MyVanna(ChromaDB_VectorStore, Ollama):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        Ollama.__init__(self, config=config)

vn = MyVanna(config={'model': 'gemma:2b'})

In [9]:
vn.connect_to_sqlite('my-database.sqlite')

In [10]:
df_ddl = vn.run_sql("SELECT type, sql FROM sqlite_master WHERE sql is not null")

for ddl in df_ddl['sql'].to_list():
  vn.train(ddl=ddl)

Adding ddl: CREATE TABLE sqlite_sequence(name,seq)


/Users/jijovaliyaveettil/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:48<00:00, 1.71MiB/s]


Adding ddl: CREATE TABLE "maintenance" (
    Maintenance_ID INTEGER PRIMARY KEY,
    Base_Station TEXT NOT NULL,
    Maintenance_Date TEXT NOT NULL,
    Technician_Name TEXT NOT NULL,
    Description TEXT,
    Duration_hours REAL NOT NULL,
    Cost REAL NOT NULL,
    FOREIGN KEY (Base_Station) REFERENCES universityconfigurations(Base_Station)
)
Adding ddl: CREATE TABLE "universityconfigurations" (
    ID INTEGER NOT NULL,
    Base_Station TEXT NOT NULL,
    Location TEXT NOT NULL,
    Bandwidth REAL NOT NULL,
    Transmission_Power REAL NOT NULL,
    Frequency_Band REAL NOT NULL,
    Coverage_Area REAL NOT NULL,
    Maximum_Capacity INTEGER NOT NULL,
    Operational_Status TEXT NOT NULL,
    Deployment_Date TEXT NOT NULL,
    PRIMARY KEY (Base_Station)
)


In [11]:
vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS my-table (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        age INT
    )
""")

# Sometimes you may want to add documentation about your business terminology or definitions.
vn.train(documentation="Our university defines base stations as BS-XXX, where XXX is the base station number.")


Adding ddl: 
    CREATE TABLE IF NOT EXISTS my-table (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        age INT
    )

Adding documentation....


'ff9ba045-1fd7-5636-83b9-669ecdf8976d-doc'

In [12]:
training_data = vn.get_training_data()
training_data

,id,question,content,training_data_type
0,40260ea4-7947-51dd-9961-8a55d48ba286-ddl,None,"CREATE TABLE ""maintenance"" (\n Maintenance_...",ddl
1,4aafa9de-6443-5409-86f9-5d581327d6a0-ddl,None,\n CREATE TABLE IF NOT EXISTS my-table (\n ...,ddl
2,4ea4e13c-6122-5334-8f14-f9d3563e9591-ddl,None,"CREATE TABLE ""universityconfigurations"" (\n ...",ddl
3,ea84418b-1a28-59b4-a1f4-2fb674208adc-ddl,None,"CREATE TABLE sqlite_sequence(name,seq)",ddl
0,ff9ba045-1fd7-5636-83b9-669ecdf8976d-doc,None,Our university defines base stations as BS-XXX...,documentation


In [14]:
vn.ask(question="How many base stations are there ?")

Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 10 is greater than number of elements in index 4, updating n_results = 4
Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1


SQL Prompt: [{'role': 'system', 'content': 'You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE "universityconfigurations" (\n    ID INTEGER NOT NULL,\n    Base_Station TEXT NOT NULL,\n    Location TEXT NOT NULL,\n    Bandwidth REAL NOT NULL,\n    Transmission_Power REAL NOT NULL,\n    Frequency_Band REAL NOT NULL,\n    Coverage_Area REAL NOT NULL,\n    Maximum_Capacity INTEGER NOT NULL,\n    Operational_Status TEXT NOT NULL,\n    Deployment_Date TEXT NOT NULL,\n    PRIMARY KEY (Base_Station)\n)\n\nCREATE TABLE "maintenance" (\n    Maintenance_ID INTEGER PRIMARY KEY,\n    Base_Station TEXT NOT NULL,\n    Maintenance_Date TEXT NOT NULL,\n    Technician_Name TEXT NOT NULL,\n    Description TEXT,\n    Duration_hours REAL NOT NULL,\n    Cost REAL NOT NULL,\n    FOREIGN KEY (Base_Station) REFERENCES universityconfigur

Insert of existing embedding ID: 03cc2f9f-405e-58ed-827f-51a25f92cf8e-sql
Add of existing embedding ID: 03cc2f9f-405e-58ed-827f-51a25f92cf8e-sql


Info: Ollama parameters:
model=gemma:2b,
options={},
keep_alive=None
Info: Prompt Content:
[{"role": "system", "content": "The following is a pandas DataFrame that contains the results of the query that answers the question the user asked: 'How many base stations are there ?'\n\nThe DataFrame was produced using this query: SELECT COUNT(*) AS num_base_stations\nFROM universityconfigurations\n\nThe following is information about the resulting pandas DataFrame 'df': \nRunning df.dtypes gives:\n num_base_stations    int64\ndtype: object"}, {"role": "user", "content": "Can you generate the Python plotly code to chart the results of the dataframe? Assume the data is in a pandas dataframe called 'df'. If there is only one value in the dataframe, use an Indicator. Respond with only Python code. Do not answer with any explanations -- just the code."}]
Info: Ollama Response:
{'model': 'gemma:2b', 'created_at': '2024-08-16T00:05:32.129398Z', 'message': {'role': 'assistant', 'content': "```python\

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Traceback (most recent call last):
  File "/Users/jijovaliyaveettil/Desktop/Research Project/SQL/venv/lib/python3.12/site-packages/vanna/base/base.py", line 1708, in ask
    img_bytes = fig.to_image(format="png", scale=2)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jijovaliyaveettil/Desktop/Research Project/SQL/venv/lib/python3.12/site-packages/plotly/basedatatypes.py", line 3772, in to_image
    return pio.to_image(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jijovaliyaveettil/Desktop/Research Project/SQL/venv/lib/python3.12/site-packages/plotly/io/_kaleido.py", line 143, in to_image
    img_bytes = scope